In [133]:
import itertools
import math
import numpy as np


# After everything is done adjust this class to make an initial guess based on flow
# values from nodes and K's until that point the main goal is to solve problem with a
# automized way. 

class Node:
  def __init__(self, name, flow):
    self._name = name
    self._flow = flow

class Member:
  id_iter = itertools.count()
  def __init__(self, initial_point, end_point, diameter = 1, length = 100, K = 1):
    self._id = next(self.id_iter)
    self._initial_point = initial_point
    self._end_point = end_point
    self._length = length
    self._K = K
    self._diameter = diameter

  def __repr__(self):
    return self._initial_point + self._end_point

  def get_id(self):
    return self._id

  def area(self):
    return math.pi * self._diameter**2 / 4
  
  def get_K(self):
    return self._K
  

class Loop:
  def __init__(self, members, discharges, directions):
    self._members = members
    self._discharges = discharges
    self._directions = directions

  def __repr__(self):
    adjusted_flow = list(np.array(self._discharges) * np.array(self._directions))
    return f"Loop members are {self._members}, and discharges are {adjusted_flow}"
  
  def discharge_rate_of_members(self):
    return list(np.array(self._discharges) * np.array(self._directions))


  def __iter__(self):
    return LoopIter(self)


  def update_discharge_rates(self, new_discharge_rates):
    self._discharges = new_discharge_rates

  def update_directions(self, new_directions):
    self._directions = new_directions
  
  def get_ids_of_members(self):
    member_id_list = []
    for member in self._members:
      member_id_list.append(member.get_id())
    return member_id_list

class LoopIter:
    def __init__(self, loop):
        self._memb = loop._members
        self._loop_size = len(self._memb)
        self._current_index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self._current_index < self._loop_size:
            member = self._memb[self._current_index] 
            self._current_index += 1
            return member

        raise StopIteration


class Network:
  def __init__(self, loops, dQ = 3):
    self._loops = loops
    self._dQ = dQ

  
  def solve_network(self):
    deltaQ = 15
    count = 1
    while abs(deltaQ) > self._dQ:
      print("Iteration: ", str(count)," ")
      for loop in self._loops:
        i = 0
        sum_KQQ = 0
        sum_2KQ = 0
        discharge_rates = loop.discharge_rate_of_members()
        for member in loop:
          discharge_rate = discharge_rates[i]
          sum_KQQ += self.K_Q_Q(member, discharge_rate)
          sum_2KQ += self.K_Q(member, discharge_rate)
          i += 1
        # print(sum_2KQ)
        deltaQ = -(sum_KQQ / sum_2KQ)
        new_discharges = []
        new_directions = []
        for i in discharge_rates:
          i += deltaQ
          new_discharges.append(abs(i))
          if i > 0:
            new_directions.append(1)
          else:
            new_directions.append(-1)
        # print(deltaQ)
        # print(new_discharges)
        loop.update_directions(new_directions)
        loop.update_discharge_rates(new_discharges)  
        # There are some common members so fix them accordingly
        other_loops = self._loops.copy()
        other_loops.remove(loop)
        for other_loop in other_loops:
          other_member_ids = other_loop.get_ids_of_members()
          # print(loop.get_ids_of_members()) # [4, 2, 3]
          # print(other_member_ids) # [0, 1, 4]
          other_discharge = []
          other_directions = []
          for i, id in enumerate(other_member_ids):
            if id in loop.get_ids_of_members():
              index_of_main_loop = loop.get_ids_of_members().index(id)
              other_discharge.append(loop._discharges[index_of_main_loop])
              other_directions.append(loop._directions[index_of_main_loop] * (-1))
              # print(loop._discharges[index_of_main_loop])
            else:
              other_discharge.append(other_loop._discharges[i])
              other_directions.append(other_loop._directions[i])
          # print(other_discharge)
          # print(other_directions)
          other_loop.update_directions(other_directions)
          other_loop.update_discharge_rates(other_discharge)
      self.final_solution(self._loops)
      print("\n")
      count += 1

    print("Final Discharge Rates in Each Member")     
    self.final_solution(self._loops)


  def K_Q_Q(self, member, Q):
    return member.get_K()*Q*abs(Q)

  def K_Q(self, member, Q):
    return 2*member.get_K()*abs(Q)

  def final_solution(self, loops):
    for loop in loops:
      print(loop)

  def common_elements(self, loop):
    ids_of_loop = loop.get_ids_of_members()

In [134]:
AB = Member("A", "B", K = 1)
BC = Member("B", "C", K = 2)
DC = Member("D", "C", K = 5)
AD = Member("A", "D", K = 6)
AC = Member("A", "C", K = 3)

loop1_elements = [AC, DC, AD]
loop2_elements = [AB, BC, AC]

loop1 = Loop(loop1_elements, [35, 30, 70], directions = [1, -1, 1])
loop2 = Loop(loop2_elements, [15, 35, 35], directions = [1, -1, -1])


In [135]:
loops = [loop1, loop2]
network = Network(loops)
network.solve_network()

Iteration:  1  
Loop members are [AC, DC, AD], and discharges are [2.7697628458498027, -51.16666666666667, 48.83333333333333]
Loop members are [AB, BC, AC], and discharges are [26.06357048748353, -23.93642951251647, -2.7697628458498027]


Iteration:  2  
Loop members are [AC, DC, AD], and discharges are [-1.3528398233467842, -52.28042325943739, 47.71957674056261]
Loop members are [AB, BC, AC], and discharges are [29.072416563909396, -20.927583436090604, 1.3528398233467842]


Iteration:  3  
Loop members are [AC, DC, AD], and discharges are [-1.5136290044266474, -52.27248910804394, 47.72751089195606]
Loop members are [AB, BC, AC], and discharges are [29.24113989638271, -20.75886010361729, 1.5136290044266474]


Final Discharge Rates in Each Member
Loop members are [AC, DC, AD], and discharges are [-1.5136290044266474, -52.27248910804394, 47.72751089195606]
Loop members are [AB, BC, AC], and discharges are [29.24113989638271, -20.75886010361729, 1.5136290044266474]
